In [1]:
import pandas as pd
import os
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)

db = client.test_database
SS_Data = db['Saber_Sim_Data']
_4f4_Proj = db['_4for4_Projection_Data']
_4f4_Ceil = db['_4for4_Ceiling_Data']
_4f4_WR_cb = db['_4for4_WR_cb']
_4f4_RedZ = db['_4for4__RedZone_Data']
_4f4_WR_fp = db['_4for4_WR_fantasy_points']
_4f4_RB_fp = db['_4for4_RB_fantasy_points']
_4f4_RB_tar = db['_4for4_RB_target']
airy_WR = db['AirY_WR']
airy_TE = db['AirY_TE']
ETR_Ceil = db['ETR_Ceiling']


In [3]:
ss_data_cols = """Name
Team
Opponent
Position
Projection
Actual
Price
Value
proj_own
dk_point
dk_25_percentile
dk_50_percentile
dk_75_percentile
dk_85_percentile
dk_95_percentile
dk_99_percentile
fd_points
fd_25_percentile
fd_50_percentile
fd_75_percentile
fd_85_percentile
fd_95_percentile
fd_99_percentile
fdraft_points
fdraft_25_percentile
fdraft_50_percentile
fdraft_75_percentile
fdraft_85_percentile
fdraft_95_percentile
fdraft_99_percentile
yahoo_points
yahoo_25_percentile
yahoo_50_percentile
yahoo_75_percentile
yahoo_85_percentile
yahoo_95_percentile
yahoo_99_percentile
dk_std
fd_std
fdraft_std
yahoo_std
pass_attempts
completions
passing_yards
passing_td
receptions
receiving_yards
receiving_td
rushes
rushing_yards
rushing_td""".split('\n')

_4f4_proj_cols = """Season
Week
PID
Player
Pos
Team
Opp
aFPA
aFPA_Rk
FFPts
Comp
Pass_Att
Pass_Yds
Pass_TD
INT
Rush_Att
Rush_Yds
Rush_TD
Rec
Rec_Yds
Rec_TD
Pa1D
Ru1D
Rec1D
Fum
XP
FG
Grade""".split('\n')

_4f4_ceil_cols = """Season
Week
PID
Player
Pos
Team
Opp
aFPA
aFPA_Rk
FFPts
Comp
Pass_Att
Pass_Yds
Pass_TD
INT
Rush_Att
Rush_Yds
Rush_TD
Rec
Rec_Yds
Rec_TD
Pa1D
Ru1D
Rec1D
Fum
XP
FG
Grade
DK_Proj
DK_Price
DK_Val
DK_Pts_per_1k
DK_Flr
DK_Flr_Val
DK_Flr_per_1k
DK_Ceil
DK_Ceil_Val
DK_Ceil_per_1k
DK_Wk_Change
DK_Own_Percent
FD_Proj
FD_Price
FD_Val
FD_Pts_pr_1k
FD_Flr
FD_Flr_Val
FD_Flr_pr_1k
FD_Ceil
FD_Ceil_Val
FD_Ceil_pr_1k
FD_Wk_Change
FD_Own_Percent
Y_Proj
Y_Price
Y_Val
Y_Pt_pr_Dollar
Y_Flr
Y_Flr_Val
Y_Flr_pr_Dollar
Y_Ceil
Y_Ceil_Val
Y_Ceil_pr_Dollar
Y_Wk_Change
FyD_Proj
FyD_Price
FyD_Val
FyD_Pt_pr_1k
FyD_Flr
FyD_Flr_Val
FyD_Flr_pr_1k
FyD_Ceil
FyD_Ceil_Val
FyD_Ceil_pr_1k
FyD_Wk_Change
O_U
Line
Team_O_U""".split('\n')

_4f4_wrcb_cols = ['Player', 'Team', 'Opponent', 'Matchup', 'Shadow']
_4f4_wr_fp_cols = ['Player', 'Pos', 'Team', 'G', 'W5', 'W6', 'W7', 'W8', 'DK_Pts','DK_Pts_pr_G']
_4f4_rb_fp_cols = ['Player', 'Pos', 'Team', 'G', 'W13', 'W14', 'W15', 'DK_Pts','DK_Pts_pr_G']
_4f4_rb_tar_cols = ['Player', 'Pos', 'Team', 'G', 'W13', 'W14', 'W15', 'TGTs', 'TGTs_pr_G', 'Array']
airy_wr_cols = ['full_name', 'position', 'team', 'targets', 'rec', 'rec_yards',
           'air_yards', 'yac', 'td', 'adot', 'racr', 'ms_air', 'tgt_share', 'wopr',
           'ppr', 'Tar_pr_Gam', 'AY_pr_Gam']
airy_te_cols = ['full_name', 'position', 'team', 'targets', 'rec', 'rec_yards',
           'air_yards', 'yac', 'td', 'adot', 'racr', 'ms_air', 'tgt_share', 'wopr',
           'ppr', 'Tar_pr_3']

_4f4_redZ_cols = ['Season', 'Week', 'PID', 'Player', 'Pos', 'Team', 'Opp', 'aFPA',
'aFPA_Rk', 'FFPts', 'Comp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'INT',
'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec', 'Rec_Yds', 'Rec_TD', 'Pa1D',
'Ru1D', 'Rec1D', 'Fum', 'XP', 'FG', 'Grade', 'DK_proj', 'DK_price',
'DK_val', 'DK_Pt_pr_1k', 'DK_Flr', 'DK_Flr_Val', 'DK_Flr_pr_1k',
'DK_Ceil', 'DK_Ceil_Val', 'DK_Ceil_pr_1k', 'DK_Wk_Change',
'DK_Own_Percent', 'FD_proj', 'FD_price', 'FD_val', 'FD_Pt_pr_1k',
'FD_Flr', 'FD_Flr_Val', 'FD_Flr_pr_1k', 'FD_Ceil',
'FD_Ceil_Val', 'FD_Ceil_pr_1k', 'FD_Wk_Change', 'FD_Own_Percent',
'Y_proj', 'Y_price', 'Y_val', 'Y_Pt_pr_DOllar', 'Y_Flr',
'Y_Flr_Val', 'Y_Flr_pr_Dollar', 'Y_Ceil', 'Y_Ceil_Val',
'Y_Ceil_pr_Dollar', 'Y_Wk_Change', 'FyD_proj', 'FyD_price', 'FyD_val',
'FyD_Pt_pr_1k', 'FyD_Flr', 'FyD_Flr_Val', 'FyD_Flr_pr_1k',
'FyD_Ceil', 'FyD_Ceil_Val', 'FyD_Ceil_pr_1k', 'FyD_Wk_Change',
'O_U', 'Line', 'Team_O_U']

etr_ceiling_cols = ['Season', 'Week', 'PID', 'Player', 'Pos', 'Team', 'Opp', 'aFPA',
'aFPA_Rk', 'FFPts', 'Comp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'INT',
'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec', 'Rec_Yds', 'Rec_TD', 'Pa1D',
'Ru1D', 'Rec1D', 'Fum', 'XP', 'FG', 'Grade', 'DK_proj', 'DK_price',
'DK_val', 'DK_Pt_pr_1k', 'DK_Flr', 'DK_Flr_Val', 'DK_Flr_pr_1k',
'DK_Ceil', 'DK_Ceil_Val', 'DK_Ceil_pr_1k', 'DK_Wk_Change',
'DK_Own_Percent', 'FD_proj', 'FD_price', 'FD_val', 'FD_Pt_pr_1k',
'FD_Flr', 'FD_Flr_Val', 'FD_Flr_pr_1k', 'FD_Ceil',
'FD_Ceil_Val', 'FD_Ceil_pr_1k', 'FD_Wk_Change', 'FD_Own_Percent',
'Y_proj', 'Y_price', 'Y_val', 'Y_Pt_pr_Dollar', 'Y_Flr',
'Y_Flr_Val', 'Y_Flr_pr_Dollar', 'Y_Ceil', 'Y_Ceil_Val',
'Y_Ceil_pr_Dollar', 'Y_Wk_Change', 'FyD_proj', 'FyD_price', 'FyD_val',
'FyD_Pt_pr_1k', 'FyD_Flr', 'FyD_Flr_Val', 'FyD_Flr_pr_1k',
'FyD_Ceil', 'FyD_Ceil_Val', 'FyD_Ceil_pr_1k', 'FyD_Wk_Change',
'O_U', 'Line', 'Team_O_U']

workbook_path = os.path.join('DFS', 'Resources', 'DraftKingsNoNotes.xlsx')

workbook = pd.ExcelFile(workbook_path)

ss_data = workbook.parse('SS Data')
_4f4_proj = workbook.parse('4f4 Proj Data')
_4f4_ceil = workbook.parse('4f4 Ceiling Data')
_4f4_wrcb= workbook.parse('4f4 WRCB')
_4f4_redZ = workbook.parse('4f4 Red Z Data')
_4f4_wr_fp = workbook.parse('4f4 WR FP Last 4 ')
_4f4_rb_fp = workbook.parse('4f4 RB FP Last 3')
_4f4_rb_tar = workbook.parse('4f4 RB TAR Last 3')
airy_wr = workbook.parse('AirY WR Last 4').drop(['Unnamed: 0'], axis=1)
airy_te = workbook.parse('AirY TE Last 3').drop(['Unnamed: 0'], axis=1)
etr_ceiling = workbook.parse('ETR Ceiling Data')

ss_data.columns = ss_data_cols
_4f4_proj.columns = _4f4_proj_cols
_4f4_ceil.columns = _4f4_ceil_cols
_4f4_wrcb.columns = _4f4_wrcb_cols
_4f4_wr_fp.columns = _4f4_wr_fp_cols
_4f4_rb_fp.columns = _4f4_rb_fp_cols
_4f4_rb_tar.columns = _4f4_rb_tar_cols
airy_wr.columns = airy_wr_cols 
airy_te.columns = airy_te_cols 
_4f4_redZ.columns = _4f4_redZ_cols
etr_ceiling.columns = etr_ceiling_cols


In [4]:
ss_dict = ss_data.T.to_dict().values()
_4f4_proj_dicts = _4f4_proj.T.to_dict().values()
_4f4_ceil_dicts = _4f4_ceil.T.to_dict().values()
_4f4_wrcb_dicts = _4f4_wrcb.T.to_dict().values()
_4f4_redZ_dicts = _4f4_redZ.T.to_dict().values()
_4f4_wr_fp_dicts = _4f4_wr_fp.T.to_dict().values()
_4f4_rb_fp_dicts = _4f4_rb_fp.T.to_dict().values()
_4f4_rb_tar_dicts = _4f4_rb_tar.T.to_dict().values()
airy_wr_dicts = airy_wr.T.to_dict().values()
airy_te_dicts = airy_te.T.to_dict().values()
etr_ceiling_dicts = etr_ceiling.T.to_dict().values()

In [5]:
ss_data_ids = SS_Data.insert_many(ss_dict).inserted_ids
_4f4_proj_ids = _4f4_Proj.insert_many(_4f4_proj_dicts).inserted_ids
_4f4_ceil_ids = _4f4_Ceil.insert_many(_4f4_ceil_dicts).inserted_ids
_4f4_wrcb_ids = _4f4_WR_cb.insert_many(_4f4_wrcb_dicts).inserted_ids
_4f4_redZ_ids = _4f4_RedZ.insert_many(_4f4_redZ_dicts).inserted_ids
_4f4_wr_fp_ids = _4f4_WR_fp.insert_many(_4f4_wr_fp_dicts).inserted_ids
_4f4_rb_fp_ids = _4f4_RB_fp.insert_many(_4f4_rb_fp_dicts).inserted_ids
_4f4_rb_tar_ids = _4f4_RB_tar.insert_many(_4f4_rb_tar_dicts).inserted_ids
airy_wr_ids = airy_WR.insert_many(airy_wr_dicts).inserted_ids
airy_te_ids = airy_TE.insert_many(airy_te_dicts).inserted_ids
etr_ceiling_ids = ETR_Ceil.insert_many(etr_ceiling_dicts).inserted_ids

In [15]:
db_ss_data = SS_Data.find_one()

In [18]:
db_ss_data

{'_id': ObjectId('5efab9f54cb5cdab023d52c7'),
 'Name': 'Christian McCaffrey',
 'Team': 'CAR',
 'Opponent': 'NO',
 'Position': 'RB',
 'Projection': 27.1773,
 'Actual': 'undefined',
 'Price': 10000,
 'Value': 2.71773,
 'proj_own': 31.8,
 'dk_point': 27.1773,
 'dk_25_percentile': 18.9,
 'dk_50_percentile': 25.85,
 'dk_75_percentile': 33.9,
 'dk_85_percentile': 38.4,
 'dk_95_percentile': 46.105,
 'dk_99_percentile': 58.602,
 'fd_points': 21.9361,
 'fd_25_percentile': 15.5,
 'fd_50_percentile': 21.1,
 'fd_75_percentile': 27.325,
 'fd_85_percentile': 30.815,
 'fd_95_percentile': 37.405,
 'fd_99_percentile': 50.001,
 'fdraft_points': 27.1773,
 'fdraft_25_percentile': 18.9,
 'fdraft_50_percentile': 25.85,
 'fdraft_75_percentile': 33.9,
 'fdraft_85_percentile': 38.4,
 'fdraft_95_percentile': 46.105,
 'fdraft_99_percentile': 58.602,
 'yahoo_points': 21.9361,
 'yahoo_25_percentile': 15.5,
 'yahoo_50_percentile': 21.1,
 'yahoo_75_percentile': 27.325,
 'yahoo_85_percentile': 30.815,
 'yahoo_95_perc